In [3]:
from paddleocr import PaddleOCR

# Inicializa OCR en inglés (alfanumérico)
ocr = PaddleOCR(lang='en', use_textline_orientation=True)

# Ruta de la imagen
path_im = r"C:\Users\javie\Documents\TRABAJO UNI\VC\Vision_por_Computador\Práctica_4\Parte_A\TGC_RBNW\train\images\0116GPD.jpg"

# Detecta y reconoce texto en la imagen completa
ocr_result = ocr.predict(path_im)

first_result = ocr_result[0]  # esto es un diccionario

# acceder directamente al texto
if 'rec_texts' in first_result and len(first_result['rec_texts']) > 0:
    texto_matricula = first_result['rec_texts'][0]
    print(texto_matricula)
else:
    texto_matricula = ""



Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\javie\.paddlex\official_models\PP-LCNet_x1_0_doc_ori`.


Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\javie\.paddlex\official_models\UVDoc`.
Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\javie\.paddlex\official_models\PP-LCNet_x1_0_textline_ori`.
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\javie\.paddlex\official_models\PP-OCRv5_server_det`.
Creating model: ('en_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\javie\.paddlex\official_models\en_PP-OCRv5_mobile_rec`.


0116 GPD


In [ ]:
import cv2  
import math 
import csv
import time
import numpy as np
from collections import Counter
from ultralytics import YOLO
import torch
from paddleocr import PaddleOCR
from PIL import Image

ocr = PaddleOCR(lang='en', use_textline_orientation=True)

model_general = YOLO('yolo11n.pt')
model_plate = YOLO(
    r"runs\detect\matricula_yolo11n\weights\best.pt"
)

classNames = ["person", "bicycle", "car"]

video_path = r"C0142.MP4"
vid = cv2.VideoCapture(video_path)

output_video_path = r"resultados_video\\resultados_deteccion.mp4"
output_csv_path = r"resultados_video\\detecciones_matriculas.csv"

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = int(vid.get(cv2.CAP_PROP_FPS))
width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

frame_count = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"Procesando video ({frame_count} fotogramas)...\n")

total_tiempo_yolo = 0
total_tiempo_ocr = 0
num_placas = 0

with open(output_csv_path, mode='w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([
        "frame", "tipo_objeto", "confianza", "id_tracking",
        "x1", "y1", "x2", "y2",
        "conf_matricula",
        "mx1", "my1", "mx2", "my2", "texto_matricula",
        "tiempo_yolo_s", "tiempo_ocr_s"
    ])

    conteo = Counter()
    frame_num = 0

    while True:
        ret, frame = vid.read()
        if not ret:
            break
        frame_num += 1

        start_yolo = time.time()
        results_general = model_general.track(frame, persist=True, classes=[0, 2],  verbose=False)
        tiempo_yolo = time.time() - start_yolo
        total_tiempo_yolo += tiempo_yolo

        if results_general and len(results_general[0].boxes) > 0:
            for box in results_general[0].boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                cls = int(box.cls[0])
                label = classNames[cls]
                track_id = int(box.id[0]) if box.id is not None else -1

                color = (0, 255, 0) if label == "person" else (255, 0, 0)
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, f"{label} {conf:.2f} ID:{track_id}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

                conteo[label] += 1
                conf_matricula = ""
                mx1 = my1 = mx2 = my2 = ""
                texto_matricula = ""
                tiempo_ocr = 0

                if label == "car":
                    car_crop = frame[y1:y2, x1:x2]
                    if car_crop.size > 0:
                        results_plate = model_plate.predict(source=car_crop, device=0, verbose=False)
                        for p in results_plate:
                            for b in p.boxes:
                                px1, py1, px2, py2 = map(int, b.xyxy[0])
                                pconf = float(b.conf[0])
                                mx1, my1, mx2, my2 = x1 + px1, y1 + py1, x1 + px2, y1 + py2
                                conf_matricula = round(pconf, 2)
                                plate_crop = car_crop[py1:py2, px1:px2]
                                if plate_crop.size > 0:
                                   

                                    start_ocr = time.time()
                                    plate_rgb = cv2.cvtColor(plate_crop, cv2.COLOR_BGR2RGB)
                                    ocr_result = ocr.predict([plate_rgb])
                                    tiempo_ocr = time.time() - start_ocr
                                    total_tiempo_ocr += tiempo_ocr
                                    num_placas += 1


                                    if ocr_result and len(ocr_result[0]) > 0:
                                        first_result = ocr_result[0]  # esto es un diccionario

                                        # acceder directamente al texto
                                        if 'rec_texts' in first_result and len(first_result['rec_texts']) > 0:
                                            texto_matricula = first_result['rec_texts'][0]
                                        else:
                                            texto_matricula = ""
                                        if texto_matricula:
                                            cv2.putText(frame, texto_matricula, (mx1, my2 + 20),
                                                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
                                cv2.rectangle(frame, (mx1, my1), (mx2, my2), (0, 255, 255), 2)
                                cv2.putText(frame, f"plate {pconf:.2f}", (mx1, my1 - 5),
                                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
                                break

                writer.writerow([
                    frame_num, label, round(conf, 2), track_id,
                    x1, y1, x2, y2,
                    conf_matricula,
                    mx1, my1, mx2, my2, texto_matricula,
                    round(tiempo_yolo, 4), round(tiempo_ocr, 4)
                ])

        out.write(frame)
        if frame_num % 10 == 0:
            print(f"Procesados {frame_num}/{frame_count} fotogramas...")

vid.release()
out.release()
cv2.destroyAllWindows()

print("\nPROCESAMIENTO COMPLETADO")
print(f"Video guardado en: {output_video_path}")
print(f"CSV guardado en: {output_csv_path}")
print(f"Conteo total: {dict(conteo)}")
if frame_count > 0:
    print(f"Tiempo medio YOLO por frame: {total_tiempo_yolo/frame_count:.4f} s")
if num_placas > 0:
    print(f"Tiempo medio OCR por matrícula: {total_tiempo_ocr/num_placas:.4f} s")


Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\javie\.paddlex\official_models\PP-LCNet_x1_0_doc_ori`.
Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\javie\.paddlex\official_models\UVDoc`.
Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\javie\.paddlex\official_models\PP-LCNet_x1_0_textline_ori`.
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\javie\.paddlex\official_models\PP-OCRv5_server_det`.
Creating model: ('en_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\javie\.paddlex\

Procesando video (2832 fotogramas)...

Procesados 10/2832 fotogramas...
Procesados 20/2832 fotogramas...
Procesados 30/2832 fotogramas...
Procesados 40/2832 fotogramas...
Procesados 50/2832 fotogramas...
Procesados 60/2832 fotogramas...
Procesados 70/2832 fotogramas...
Procesados 80/2832 fotogramas...
Procesados 90/2832 fotogramas...
Procesados 100/2832 fotogramas...
Procesados 110/2832 fotogramas...


ValueError: axes don't match array